In [1]:
import requests
from bs4 import BeautifulSoup

In [ ]:
def Wikipedia_search(address):
    try:
        req = requests.get(address,timeout=5)
    except:
        return "Timeout error.."
    source = req.text
    soup = BeautifulSoup(source, 'html.parser')

    soup_info = soup.select("#mw-content-text > div.mw-parser-output")
    index_info = soup.select("#toc")
    infobox_info = soup.select("#mw-content-text > div.mw-parser-output > table.infobox.vcard")
    ref_info = soup.select("#mw-content-text > div.mw-parser-output > div.reflist")

    filtered_info = soup_info[0].text
    if(len(index_info)>0):
        filtered_info = filtered_info.replace(index_info[0].text,"")
    if(len(infobox_info)>0):
        filtered_info = filtered_info.replace(infobox_info[0].text,"")
    if(len(ref_info)>0):
        filtered_info = filtered_info.replace(ref_info[0].text,"")

    comp_data = str(soup_info[0])
    index = 0
    table_list = []
    while index > -1:
        index = comp_data.find('<table class=', index)
        if index > -1:
            table_info = str(str(soup_info[0])[index:comp_data.find('>', index)])
            table_list.append(table_info.split("=")[1].split("\"")[1].split(" ")[0])
            index += len('<table class="')


    table_result = list(map(lambda x: soup.select("."+str(x)), list(set(table_list))))

    for table_index in range(len(table_result)):
        for table_buffer in range(len(table_result[table_index])):
            filtered_info = filtered_info.replace(str(table_result[table_index][table_buffer].text),"")

    ## Ref number 삭제
    for N_ref in list(map(lambda x: x.text, soup.select('.reference'))):
        filtered_info = filtered_info.replace(N_ref,"")

    ## Reference 이후 불필요한 부분 삭제
    index = 0
    table_list = []
    while index > -1:
        index = filtered_info.find("References[edit]", index)
        if index > -1:
            filtered_info = filtered_info[:index]
            index += len('<table class="')

    filtered_info = filtered_info.replace("[edit]","") ## edit 제거
    
    filtered_info = filtered_info.split("\n")

    strip_list = list(map(lambda x: x.strip(),filtered_info))
    #print(strip_list)

    sentence = ""
    for ingredient in strip_list:
        if(ingredient == ''):
            pass
        else:
            ingredient = ingredient.replace("\xa0"," ") ## 이따금 들어오는 공백 널문자는 공백으로 대체함.
            ## 각 요소 마다 마지막엔 구두점으로 끝나게하자.(문장 구분을 통해 NLP성능 극대화)
            if(ingredient[-1] == "."):
                pass
            else:
                ingredient += "."
            sentence += ingredient + " "
    
    return sentence[:-1]